In [3]:
import os
import logging
from pprint import pprint

import pandas as pd
from extract_urls import get_hyperlinks
import openai
from dotenv import load_dotenv
from langchain.document_loaders import WebBaseLoader, DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv('openai.env')
logger = logging.getLogger()
openai.api_key  = os.getenv('OPENAI_API_KEY')

"""
Создать файл openai.env, содержащий:
OPENAI_API_KEY="ключь GPT"
"""

'\nСоздать файл openai.env, содержащий:\nOPENAI_API_KEY="ключь GPT"\n'

# Splitting

## Pandas loader 

In [ ]:
df = pd.read_csv('test.csv')
df['page_content'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
loader = DataFrameLoader(df, page_content_column="page_content")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = df.shape[0], chunk_overlap = 1)
all_splits = text_splitter.split_documents(data)
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory='langchain_embeddings')

# Question Answerning

In [68]:
template = """ На вход подаются данные о событиях, которые произошли в процессе сделки. 
Описание данных:
type - это тип события. Вот основные типы, которые может принят это поле:\
    initial - инициализация события, \
    sale_field_changed - изменение цены, \
    entity_responsible_changed - менеджер ответственный за сделку изменен,\
    common_note_added - добавлено примечание о сделке
    entity_linked - добавили/изменили компанию или контакт \n
created_by - это имя менеджера создатея сделки, \n
created_at - дата, когда произошло событие\n
specific_data - это специфические данные, которые могут менятьс я в зависимости от поля type,\n
client - имя клиента,\n
company - название компании,\n
sale - цена (бюджет) сделки,\n
lead_status - статус сделки, \n
pipline - название воронки сделки,\n
responsible - имя менеджера, ответственного за сделку.

###############################################################################
Ответить на вопрос, основываясь только на данных. Расслабся и никуда не спеши:
{context}

Вопрос: {question}
"""

exstract_field = """Проанализируй запрос. Если в нем упоминается имя менеджера"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
retrieval_chain.invoke("Расскажи о работе всех менеджеров, которые участвовали в сделке?")

'Для ответа на данный вопрос необходимо проанализировать данные о событиях, которые произошли в процессе сделки. В каждом событии указано имя менеджера, который создал событие (поле created_by) и имя менеджера, ответственного за сделку (поле responsible).\n\nИсходя из предоставленных данных, можно выделить следующих менеджеров, которые участвовали в сделке:\n\n1. Менеджер, создавший сделку: bulatovbogdann@gmail.com\n2. Менеджер, ответственный за сделку: bulatovbogdann@gmail.com\n\nТаким образом, в сделке участвовал только один менеджер - bulatovbogdann@gmail.com.'